## 研究能不能直接将源码中的HAN用openhgnn的HAN来实现

In [2]:
import os
import warnings

import torch

warnings.filterwarnings("ignore")

os.environ["DGLBACKEND"] = "pytorch"

import dgl
import torch.nn as nn
from openhgnn.dataset.NodeClassificationDataset import OHGB_NodeClassification
from openhgnn.utils import extract_metapaths, get_ntypes_from_canonical_etypes

In [3]:
acm = OHGB_NodeClassification(
    dataset_name="ohgbn-acm", raw_dir="./dataset", logger=None
)

Extracting file to ./openhgnn/dataset\ohgbn-acm
Done saving data into cached files.


In [4]:
# 数据集获得的hg和meta_paths_dict，即用户输入
hg = acm.g

In [5]:
ntypes = get_ntypes_from_canonical_etypes(hg.canonical_etypes)
ntype_meta_paths_dict = {}
for ntype in ntypes:
    ntype_meta_paths_dict[ntype] = {}
    for meta_path_name, meta_path in acm.meta_paths_dict.items():
        if meta_path[0][0] == ntype:
            ntype_meta_paths_dict[ntype][meta_path_name] = meta_path
print("原来就有的metapath")
print(ntype_meta_paths_dict)

print("\n加上extract的metapath")
for ntype, meta_paths_dict in ntype_meta_paths_dict.items():
    if len(meta_paths_dict) == 0:
        ntype_meta_paths_dict[ntype] = extract_metapaths(ntype, hg.canonical_etypes)
print(ntype_meta_paths_dict)

原来就有的metapath
{'paper': {'PAP': [('paper', 'paper-author', 'author'), ('author', 'author-paper', 'paper')], 'PSP': [('paper', 'paper-subject', 'subject'), ('subject', 'subject-paper', 'paper')]}, 'subject': {}, 'author': {}}

加上extract的metapath
{'paper': {'PAP': [('paper', 'paper-author', 'author'), ('author', 'author-paper', 'paper')], 'PSP': [('paper', 'paper-subject', 'subject'), ('subject', 'subject-paper', 'paper')]}, 'subject': {'mp0': [('subject', 'subject-paper', 'paper'), ('paper', 'paper-subject', 'subject')]}, 'author': {'mp0': [('author', 'author-paper', 'paper'), ('paper', 'paper-author', 'author')]}}


In [5]:
in_dim = 1902
hidden_dim = 512
out_dim = 256
dropout = 0.5
layer_num_heads = 1

In [6]:
import torch.nn.functional as F
from dgl.nn import GATConv
from openhgnn.layers import MetapathConv
from openhgnn.layers.macro_layer.SemanticConv import SemanticAttention
from openhgnn.models import HAN

openhgnn中HAN里的HANLayer
```python
class HANLayer:
    def __init__(self, meta_paths_dict, in_dim, out_dim, layer_num_heads, dropout):
        super(HANLayer, self).__init__()
        self.meta_paths_dict = meta_paths_dict
        semantic_attention = SemanticAttention(in_size=out_dim * layer_num_heads)
        mods = nn.ModuleDict({mp: GATConv(in_dim, out_dim, layer_num_heads,
                                          dropout, dropout, activation=F.elu,
                                          allow_zero_in_degree=True) for mp in meta_paths_dict})
        self.model = MetapathConv(meta_paths_dict, mods, semantic_attention)
        self._cached_graph = None
        self._cached_coalesced_graph = {}

    def forward(self, g, h):
        # For full batch, it is a heterogeneous graph.
        # For mini batch, it is a dict from mata path name to DGLBlock.

        # mini batch
        if isinstance(g, dict):
            h = self.model(g, h)
        
        # full batch
        else:
            if self._cached_graph is None or self._cached_graph is not g:
                self._cached_graph = g
                self._cached_coalesced_graph.clear()
                for mp, mp_value in self.meta_paths_dict.items():
                    self._cached_coalesced_graph[mp] = dgl.metapath_reachable_graph(
                        g, mp_value)
            h = self.model(self._cached_coalesced_graph, h)
        
        return h
```

`if self._cached_graph is None or self._cached_graph is not g:`，这一步判断了g是否是以前的g，说明可以根据这个来改变masked_gs






In [7]:
mods = nn.ModuleDict(
    {
        mp: GATConv(
            in_dim,
            out_dim,
            layer_num_heads,
            dropout,
            dropout,
            activation=F.elu,
            allow_zero_in_degree=True,
        )
        for mp in acm.meta_paths_dict
    }
)
semantic_attention = SemanticAttention(in_size=out_dim * layer_num_heads)

In [8]:
# HAN的forward中，要将hg根据mp切片为DGLBlock

_cached_coalesced_graph={}
for mp_name, mp in acm.meta_paths_dict.items():
    _cached_coalesced_graph[mp_name]=dgl.metapath_reachable_graph(hg,mp)

# 这一步将hg转换为了dict{metapath_name:mp-based graph(DGLBlock)}
print(_cached_coalesced_graph['PAP'])

g_dict=_cached_coalesced_graph

Graph(num_nodes=3025, num_edges=29436,
      ndata_schemes={'pspap_m2v_emb': Scheme(shape=(64,), dtype=torch.float32), 'psp_m2v_emb': Scheme(shape=(64,), dtype=torch.float32), 'pap_m2v_emb': Scheme(shape=(64,), dtype=torch.float32), 'train_mask': Scheme(shape=(), dtype=torch.uint8), 'test_mask': Scheme(shape=(), dtype=torch.uint8), 'label': Scheme(shape=(), dtype=torch.int64), 'h': Scheme(shape=(1902,), dtype=torch.float32), 'valid_mask': Scheme(shape=(), dtype=torch.uint8)}
      edata_schemes={})


HAN中使用到了MetapathConv
```python
class MetapathConv(nn.Module):
    def __init__(self, meta_paths_dict, mods, macro_func, **kargs):
        super(MetapathConv, self).__init__()
        # One GAT layer for each meta path based adjacency matrix
        self.mods = mods
        self.meta_paths_dict = meta_paths_dict
        self.SemanticConv = macro_func

    def forward(self, g_dict, h_dict):
        outputs = {g.dsttypes[0]: [] for s, g in g_dict.items()}

        for meta_path_name, meta_path in self.meta_paths_dict.items():
            new_g = g_dict[meta_path_name]

            # han minibatch
            if h_dict.get(meta_path_name) is not None:
                h = h_dict[meta_path_name][new_g.srctypes[0]]
            # full batch
            else:
                h = h_dict[new_g.srctypes[0]]
            outputs[new_g.dsttypes[0]].append(self.mods[meta_path_name](new_g, h).flatten(1))
        # semantic_embeddings = th.stack(semantic_embeddings, dim=1)  # (N, M, D * K)
        # Aggregate the results for each destination node type
        rsts = {}
        for ntype, ntype_outputs in outputs.items():
            if len(ntype_outputs) != 0:
                rsts[ntype] = self.SemanticConv(ntype_outputs)  # (N, D * K)
        return rsts
```

In [9]:
with torch.no_grad():
    outputs = {g.dsttypes[0]: [] for s, g in g_dict.items()}
    h_dict=hg.ndata["h"]
    for meta_path_name, meta_path in acm.meta_paths_dict.items():
        new_g = g_dict[meta_path_name]
    
        # han minibatch
        if h_dict.get(meta_path_name) is not None:
            h = h_dict[meta_path_name][new_g.srctypes[0]]
        # full batch
        else:
            h = h_dict[new_g.srctypes[0]]
        outputs[new_g.dsttypes[0]].append(mods[meta_path_name](new_g, h).flatten(1))
    print(outputs)
    print()
    # outputs是一个dict，属于paper的h，两条路径的聚合
    rsts = {}
    for ntype, ntype_outputs in outputs.items():
        if len(ntype_outputs) != 0:
            rsts[ntype] = semantic_attention(ntype_outputs)  # (N, D * K)
    
    print(rsts)

{'paper': [tensor([[ 1.2369e-01,  2.1195e-01,  6.5390e-02,  ...,  5.0411e-01,
          6.5650e-02,  5.9255e-02],
        [ 9.3733e-02,  3.3919e-01, -1.2487e-01,  ..., -6.4489e-02,
          5.9769e-02,  1.0934e-03],
        [-3.1773e-03,  3.4519e-01, -2.1395e-01,  ...,  7.7881e-01,
         -6.4730e-01,  3.3984e-01],
        ...,
        [-5.9106e-02,  1.7093e-01, -1.5496e-01,  ...,  2.7477e-01,
         -3.6291e-01,  1.0742e-02],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 1.5397e+00,  1.2451e+00, -5.2654e-01,  ..., -7.6962e-01,
         -5.4486e-01,  6.1737e-01]]), tensor([[ 0.3007, -0.3414,  0.2625,  ..., -0.2552,  0.5438,  0.6403],
        [ 0.1365, -0.3032,  0.1592,  ...,  0.0596, -0.1617,  0.0055],
        [ 0.0649, -0.3340,  0.0734,  ...,  0.0293, -0.0823,  0.0672],
        ...,
        [-0.0101, -0.2234,  0.1407,  ...,  0.0283,  0.0506,  0.0115],
        [ 0.0559, -0.3819,  0.0848,  ...,  0.0674, -0.0948,  0.

In [10]:
# 直接使用MetapathConv
mpconv = MetapathConv(acm.meta_paths_dict, mods, semantic_attention)
with torch.no_grad():
    out=mpconv(g_dict, hg.ndata["h"])
out

{'paper': tensor([[-0.1914, -0.1095, -0.2157,  ...,  0.1740,  0.3112,  0.0530],
         [ 0.0835, -0.0785,  0.0708,  ...,  0.0902, -0.2457, -0.0730],
         [-0.0413, -0.2275,  0.0222,  ...,  0.1695, -0.0542, -0.0831],
         ...,
         [-0.1407, -0.1075,  0.1988,  ...,  0.2088,  0.2244, -0.0304],
         [-0.0643, -0.1892,  0.0636,  ...,  0.9898, -0.4565,  0.1926],
         [ 0.0305, -0.1768,  0.0645,  ...,  0.0158, -0.0709,  0.0248]])}

In [15]:
out['paper'].shape

torch.Size([3025, 256])

直接使用HAN

In [11]:
# openhgnn的HAN需要ntype_meta_paths_dict
ntypes = get_ntypes_from_canonical_etypes(hg.canonical_etypes)



ntype_meta_paths_dict = {}
for ntype in ntypes:
    ntype_meta_paths_dict[ntype] = {}
    for meta_path_name, meta_path in acm.meta_paths_dict.items():
        if meta_path[0][0] == ntype:
            ntype_meta_paths_dict[ntype][meta_path_name] = meta_path
print(ntype_meta_paths_dict)

{'author': {}, 'paper': {'PAP': [('paper', 'paper-author', 'author'), ('author', 'author-paper', 'paper')], 'PSP': [('paper', 'paper-subject', 'subject'), ('subject', 'subject-paper', 'paper')]}, 'subject': {}}


In [16]:
han=HAN(ntype_meta_paths_dict,in_dim,hidden_dim,out_dim,[1],dropout=0.5)


In [21]:
with torch.no_grad():
    out=han.forward(hg,h_dict)
    print(out)
    print(out['paper'].shape)

{'paper': tensor([[ 0.1791, -0.0815, -0.1062,  ...,  0.1575,  0.0690,  0.0660],
        [ 0.1339, -0.0788,  0.0339,  ...,  0.1558,  0.0770,  0.0683],
        [ 0.2612,  0.0139, -0.1024,  ...,  0.1158,  0.0659,  0.0698],
        ...,
        [ 0.2748,  0.0620, -0.2164,  ...,  0.1530,  0.1074,  0.0810],
        [ 0.3924,  0.0947, -0.0461,  ...,  0.0500,  0.0843,  0.0720],
        [ 0.1138, -0.0535,  0.0126,  ...,  0.0484,  0.0270,  0.0715]])}
torch.Size([3025, 256])
